In [1]:
# !wget https://message-from-space.readthedocs.io/en/latest/_downloads/822b494d13eb35a97f159e041a004fa9/radio-transmission-recording.wav

In [2]:
# from scipy.io import wavfile
# rate, data = wavfile.read('radio-transmission-recording.wav')

In [172]:
def pp_rows(rows):
    def conv(row):
        return ''.join("#" if c==1 else '.' for c in row)

    for row in rows:
        print(conv(row))

def pp_cols(cols):
    height = len(cols)
    width = len(cols[0])
    rows = [ [cols[r][c] for r in range(height) ] for c in range(width)]
    pp_rows(rows)


In [168]:
# f, t, sxx = signal.spectrogram(data, fs=rate, nperseg=256, window='hann') # nperseg=周波数の分割数=1024 - 44100を1024で割るから44Hz刻みぐらい? ;; 1024だと800ぐらいまで、512にすると1600まで出る
# print(f.shape) # (513,) Array of sample frequencies.
# print(t.shape) # (3041,) Array of segment times.
# print(sxx.shape) # (513, 3041) Spectrogram of x. By default, the last axis of Sxx corresponds to the segment times.

# import matplotlib.pyplot as plt
# %matplotlib inline  

from scipy.io import wavfile
from scipy import signal

# def pp_rows(rows):
#     def conv(row):
#         return ''.join("#" if c==1 else '.' for c in row)

#     for row in rows:
#         print(conv(row))
    
class SignalReader :
    def __init__(self, wav_file):
        self.rate, self.audio_data = wavfile.read('radio-transmission-recording.wav')
        self.f, self.t, self.sxx = signal.spectrogram(self.audio_data, fs=self.rate, nperseg=256, window='hann') # nperseg=周波数の分割数=1024 - 44100を1024で割るから44Hz刻みぐらい? ;; 1024だと800ぐらいまで、512にすると1600まで出る

        print("rate=", rate) # 44100
#         f, t, sxx = signal.spectrogram(data, fs=rate, nperseg=256, window='hann') # nperseg=周波数の分割数=1024 - 44100を1024で割るから44Hz刻みぐらい? ;; 1024だと800ぐらいまで、512にすると1600まで出る
        print('f:', self.f.shape) # (513,) Array of sample frequencies.
        print('t:', self.t.shape) # (3041,) Array of segment times.
        print('sxx:', self.sxx.shape) # (513, 3041) Spectrogram of x. By default, the last axis of Sxx corresponds to the segment times.

        # seg, length = sxx.shape
        # f_unit = f[1] - f[0]
        
        lo, hi = self.find_peak_indices(self.sxx)
        # mid = (f[lo]+ f[hi])/2
        of = self.onefreq(self.sxx, lo, hi)
        of12 = self.onetwo(of)

        self.data = rle(of12)
        width = self.find_width(self.data)
        print('width =', width)
        self.rows = self.box(self.data, width)

    def find_peak_indices(self, sxx):
        average = np.average(sxx, axis=1)
        # av = np.average(av1)

        indices = average.argsort()[-2:]
        return sorted(indices)  # lo, hi
    
    def onefreq(self, sxx, lo, hi):
        cutoff = np.average(sxx[hi+1:, :])
        # print("cutoff = ", cutoff)

        one = np.zeros(length)
        two = np.zeros(length)
        total = 0
        for i in range(length):
            freq = 0
            denom = 0
            for ix in range(lo-4, hi+5):
                f = ix * f_unit
                v = max(0, sxx[ix,i] - cutoff)
                freq += f * v
                denom += v
            if denom == 0:
                one[i] = 0
                two[i] = 0
            else:
                one[i] = freq / denom
                two[i] = denom
                total += denom

        avg = total / length
        # print("avg=", avg)

        threshold = avg / 5
        for i in range(length):
            if one[i] > 0 and two[i] < threshold:
                one[i] = 0
        return one

    def onetwo(self, of):
        out = np.zeros(length, dtype=int)
        m = 50
        for i in range(length):
            if 500-m < of[i] < 500+m:
                out[i] = 1
            elif 600-m < of[i] < 600+m:
                out[i] = 2
            else:
                out[i] = 0
        return out

    def rle(self, of):
        res = []
        def flush(last, cnt):
            if last > 0:
                c = round(cnt/9.8)
                return [last-1] * c
            else:
                return []
                # res.append((last-1, c))

        last = -1
        cnt = 0
        for i in range(length):
            if of[i] == last:
                cnt += 1
            else:
                if cnt > 0:
                    res += flush(last, cnt)
                last = of[i]
                cnt = 1
        if cnt > 0:
            res += flush(last, cnt)

        return res

    def find_width(self, data):
        L = len(data)
        for i in range(L):
            if data[i] == 0:
                return i-1
        return -1

    def box(self, data, width, remove_frame=True):
        L = len(data)
        height = (L + width - 1) // width
        rows = []
        if width * height > L:
            data += [0] * (width * height - L)
        for c in range(height):
            row = data[c*width : (c+1)*width]
            if remove_frame:
                if c == 0 or c == height-1:
                    continue
                row = row[1:-1]
                assert len(row) == width-2

            rows.append(row)
            # print(conv(row))
        return rows

#     def conv(self, row):
#         return ''.join("#" if c==1 else '.' for c in row)

    def show(self):
        pp_rows(self.rows)
#         for row in self.rows:
#             print(self.conv(row))

In [ ]:
# seg, length = sxx.shape
# f_unit = f[1] - f[0]

# import numpy as np

# # a = np.sum(sxx,)

# #plt.figure(figsize=(50, 10))
# #window = 3041
# #window = 400 # 50で約1って何？ 音源は約1分で、timeframeは3041、ってことは50フレームで1秒。なんでそうなる？
# #pre = 130# 140
# # plt.pcolormesh(t[pre:pre+window], f[10:20], sxx[10:20, pre:pre+window], shading='gouraud')
# # plt.show()

In [161]:
sr = SignalReader('radio-transmission-recording.wav')
# sr.show()

rate= 44100
f: (129,)
t: (12166,)
sxx: (129, 12166)
width = 24


In [162]:
#sr.rows
sr.show()

......................
..#...................
.#....................
......................
......................
..#..#................
.##..#................
......................
......................
..##..#.#.............
.#.#..#.#.............
.#....................
......................
......................
..##..#.#.#...........
.###..#.#.#...........
.#....................
......................
......................
..##..#.#.#.#.........
.#....#.#.#.#.........
.##...................
......................
......................
..##..#.#.#.#.#.......
.##...#.#.#.#.#.......
.##...................
......................
......................
..##..#.#.#.#.#.#.....
.#.#..#.#.#.#.#.#.....
.##...................
......................
......................
..##..#.#.#.#.#.#.#...
.###..#.#.#.#.#.#.#...
.##...................
......................
......................
..##..#.#.#.#.#.#.#.#.
.#....#.#.#.#.#.#.#.#.
.#.#..................
......................
...........

In [179]:
def read_number(chunk, H, W):
    assert H >= 2 and W >= 2 and chunk[0][0] == 0
    val = 0
    m = 1
    if H == W:
        # positive
        for r in range(1,H):
            for c in range(1,W):
                val += m * chunk[c][r]
                m *= 2
        return val  
    elif H == W+1:
        # negative
        return -1
    else:
        return None
    
def read_chunk(chunk):
    H = len(chunk)
    W = len(chunk[0])

    # print(' read_chunk(%dx%d)' % (H, W))
    # pp_cols(chunk)

    if chunk[0][0] == 0:
        return read_number(chunk, H, W)
    
    return None

def read_bunch(bunch, ignore_unknown=True):
    # print('')
    # print('read_bunch')
    # pp_rows(bunch)
    
    W = len(bunch)
    if W == 0:
        return []

    L = len(bunch[0])
    res = []
    chunk = []
    for c in range(L):
        col = [bunch[r][c] for r in range(W)]
        s = sum(col)
        if s == 0: # empty
            if chunk:
                res.append( read_chunk(chunk) )
                chunk = []
        else:
            chunk.append(col)
    if chunk:
        res.append( read_chunk(chunk) )

    if ignore_unknown:
        res = [item for item in res if item is not None]

    return res

def parse(rows):
    # print(rows)
    R = len(rows)
    res = []
    bunch = []
    for i in range(R):
        s = sum(rows[i])
        if s == 0: # empty
            if bunch:
                res.append(read_bunch(bunch))
                bunch = []
        else:
            bunch.append( rows[i] )
    if bunch:
        res.append(read_bunch(bunch))
    return res

p = parse(sr.rows)
# sr.show(p) #print(p)
#for row in p:
#    print(sr.conv(row))
print(p)


[[0], [1], [2], [3], [4], [5], [6], [7], [8], []]


In [193]:
#from scipy.misc import imread
from imageio import imread

def read_from_png(png):
    im = imread(png)
    # print(im.shape)
    # [H * W * 4:[r,g,b,a]]
    H, W, _ = im.shape
    H >>= 2
    W >>= 2
#     print('H=%d W=%d' % (H,W))
    d = np.zeros((H, W), dtype=int)
    for r in range(H):
        for c in range(W):
            d[r,c] = im[4*r+2, 4*c+2, 0] >= 128
    return d

d = read_from_png('img/message2.png')
print(d)

[[1 1 1 ... 1 1 1]
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 ...
 [1 0 0 ... 0 0 1]
 [1 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 1]]
